## Naive Bayes
#### Lew Sears

In [7]:
import numpy as np
import pandas as pd

In [5]:
class NaiveBayesClassifier:
    
    def __init__(self, k):
        self.k = k
    
    
    

In [12]:
sample_df = np.random.binomial(1, 0.5, size = (1000,2))
row_sum = np.sum(sample_df, axis = 1)

In [16]:
adder = lambda t: t[0]+t[1]

In [ ]:
adder